In [2]:
args <- commandArgs(T) 

In [4]:
library(here)
library(rjson)
library(Matrix)
library(tidyverse)
library(dplyr)
library(DropletUtils) 

library(Seurat)
library(Signac)
library(SeuratDisk)

library(rhdf5)
library(anndata)

# convert H5Seurat

## Multiome

In [2]:
input_path <- "/home/wsg/BM/data/HWBC/RawData"
output_path <- "/home/wsg/BM/data/HWBC/RNA+ADT/RawData"

In [3]:
HWBC_multi <- LoadH5Seurat(file = paste0(input_path, "/multi.h5seurat"))

Validating h5Seurat file

Initializing ADT with data

Adding counts for ADT

Adding variable feature information for ADT

Adding miscellaneous information for ADT

Initializing SCT with data

Adding counts for SCT

Adding variable feature information for SCT

Adding miscellaneous information for SCT

Adding reduction apca

Adding cell embeddings for apca

Adding feature loadings for apca

Adding miscellaneous information for apca

Adding reduction aumap

Adding cell embeddings for aumap

Adding miscellaneous information for aumap

Adding reduction pca

Adding cell embeddings for pca

Adding feature loadings for pca

Adding miscellaneous information for pca

Adding reduction spca

Adding cell embeddings for spca

Adding feature loadings for spca

Adding miscellaneous information for spca

Adding reduction umap

Adding cell embeddings for umap

Adding miscellaneous information for umap

Adding reduction wnn.umap

Adding cell embeddings for wnn.umap

Adding miscellaneous information for w

In [6]:
metadata <- HWBC_multi@meta.data

In [9]:
mat <- HWBC_multi@assays$SCT@counts

In [10]:
mat

  [[ suppressing 32 column names ‘L1_AAACCCAAGAAACTCA’, ‘L1_AAACCCAAGACATACA’, ‘L1_AAACCCACAACTGGTT’ ... ]]

  [[ suppressing 32 column names ‘L1_AAACCCAAGAAACTCA’, ‘L1_AAACCCAAGACATACA’, ‘L1_AAACCCACAACTGGTT’ ... ]]

  [[ suppressing 32 column names ‘L1_AAACCCAAGAAACTCA’, ‘L1_AAACCCAAGACATACA’, ‘L1_AAACCCACAACTGGTT’ ... ]]



20729 x 161764 sparse Matrix of class "dgCMatrix"
                                                                                              
AL627309.1    .  .  .  .  .  .  .  .  .  .  .  .  .  .   .  .  .  .  .  .  .  .  .  .  .  .  .
AL669831.5    1  .  .  .  .  .  .  .  .  .  .  .  .  .   .  .  .  .  .  .  .  .  .  .  .  .  .
LINC00115     .  .  .  .  .  .  .  .  .  .  .  .  .  .   .  .  .  .  .  .  .  .  .  .  .  .  .
FAM41C        .  .  .  .  1  .  .  .  .  .  .  .  .  .   .  .  .  .  .  .  .  .  .  .  .  .  .
NOC2L         .  .  .  1  .  .  .  .  .  .  .  .  .  .   .  .  .  .  1  .  1  .  .  .  .  .  .
KLHL17        .  .  .  .  .  .  .  .  .  .  .  .  .  .   .  .  .  .  .  .  .  .  .  .  .  .  1
PLEKHN1       .  .  .  .  .  .  .  .  .  .  .  .  .  .   .  .  .  .  .  .  .  .  .  .  .  .  .
AL645608.8    .  .  .  .  .  .  .  .  .  .  .  .  .  .   .  .  .  .  .  .  .  .  .  .  .  .  .
HES4          .  .  .  .  .  .  2  .  .  .  .  .  .  .   .  .  .  .  .  .  .  .  .  .  .  .  .


In [20]:
# metadata
metadata$batch <- paste0(metadata$donor, "_", metadata$time)
metadata$barcode <- rownames(metadata)
# metadata
write_csv(metadata, here(output_path, "metadata.csv"))

In [11]:
RNA_subset_counts <- mat

In [21]:
process = "raw"

# save raw rna to mtx
data_path <- here(output_path, "HWBC-CITE_seq-raw-RNA-counts.mtx")
write10xCounts(x = RNA_subset_counts, path = data_path, version = "3")

# save raw rna to rds
saveRDS(RNA_subset_counts, 
        file = here(output_path, "HWBC-CITE_seq-raw-RNA-counts.rds"))

# Create Seurat Object
RNA_subset <- CreateSeuratObject(counts = RNA_subset_counts, meta.data = metadata)

# save Seurat to h5Seurat
SaveH5Seurat(RNA_subset, overwrite = TRUE, 
             filename = here(output_path, "HWBC-CITE_seq-raw-RNA-counts.h5Seurat"))

# Convert h5Seurat to h5ad
setwd(output_path)
Convert(here(output_path, "HWBC-CITE_seq-raw-RNA-counts.h5Seurat"), dest = "h5ad")

Creating h5Seurat file for version 3.1.5.9900

Adding counts for RNA

Adding data for RNA

No variable features found for RNA

No feature-level metadata found for RNA

Validating h5Seurat file

Adding data from RNA as X

Adding counts from RNA as raw

Transfering meta.data to obs



In [ ]:
# ADT

In [22]:
mat <- HWBC_multi@assays$ADT@counts

In [23]:
mat

  [[ suppressing 33 column names ‘L1_AAACCCAAGAAACTCA’, ‘L1_AAACCCAAGACATACA’, ‘L1_AAACCCACAACTGGTT’ ... ]]



228 x 161764 sparse Matrix of class "dgCMatrix"
                                                                            
CD39           66    4    6   5    4   3  30  82   6    3    5    5    8  25
Rat-IgG1-1     15   13   19   5   10  11  19  56   5   10   12    3    8  17
CD107a         37    9    8   6   14  31  36  20   8   10    9   10   17  13
CD62P         142   66   94  79  115 117 463  97  82  106   88   66   80  72
TCR-2           4   34   10  10   10  39   6  17   2   19   48   15   21  16
CD30            3    3    9   .    4   6   6  36   3    2    3    1    5   2
CD31          242   44  103  60  169  78 624  83  21   92   40   39   41  83
CD34            9    5    4   7    6   3  18  28   8    9    6   11    7   5
CD35          752   57   60  56   73  56 228 808  46   52   77   54   69 250
CD36           14    .    2   1    5   .  23   3   2    1    2    4    1   3
CD223           5    8    6  11    4   6   4  10   6    4    3    2    4   3
TIGIT           3    2    1 

In [24]:
ADT_subset_counts <- mat

In [25]:
# save raw ADT to mtx
data_path <- here(output_path, "HWBC-CITE_seq-raw-ADT-counts.mtx")
write10xCounts(x = ADT_subset_counts, path = data_path, version = "3")

# save raw ADT to rds
saveRDS(ADT_subset_counts, 
        file = here(output_path, "HWBC-CITE_seq-raw-ADT-counts.rds"))

# Create Seurat Object
ADT_subset <- CreateSeuratObject(counts = ADT_subset_counts, meta.data = metadata)

# save Seurat to h5Seurat
SaveH5Seurat(ADT_subset, overwrite = TRUE, 
             filename = here(output_path, "HWBC-CITE_seq-raw-ADT-counts.h5Seurat"))

# Convert h5Seurat to h5ad
setwd(output_path)
Convert(here(output_path, "HWBC-CITE_seq-raw-ADT-counts.h5Seurat"), dest = "h5ad")

Creating h5Seurat file for version 3.1.5.9900

Adding counts for RNA

Adding data for RNA

No variable features found for RNA

No feature-level metadata found for RNA

Validating h5Seurat file

Adding data from RNA as X

Adding counts from RNA as raw

Transfering meta.data to obs

